In [1]:
import torch

from utils.data import Data
from ner_model import build_model, evaluate, batchify_with_label
import time
import os
import spacy

# Some standard imports
import io
import numpy as np

from torch import nn
import torch.onnx


%load_ext autoreload
%autoreload 2

In [2]:
path2xpt = '../pretrained/baseline.xpt'
path2model = '../pretrained/baseline.model'
path2dev = '../../data/aphp_cr/dev.conll'
decode_config_dict = {'load_model_dir':path2model, # load model file
                      'raw_dir': path2dev
                     }
data = Data()
#data.read_config(decode_config_dict)
print("NER MODEL: decoding-style loading..")
## dset_dir must only contains dictionnary informations here (dset from the original model should be cleaned with the function clean_dset (to be coded))
data.load_export(path2xpt)
## supplementary configurations (optional, maybe not useful in deployment)
data.read_config(decode_config_dict)
## !!! we should be loading the weights here and not at each prediction!!!!
data.HP_gpu = torch.cuda.is_available()
data.generate_instance('raw')
#data.show_data_summary()
model = build_model(data)

NER MODEL: decoding-style loading..
Load Model weights from file ../pretrained/baseline.model
building Network..
use crf:  True
use_char:  True
char feature extractor:  CNN
word feature extractor:  LSTM
Build word sequence feature extractor: LSTM...
Build word representation...
Build char sequence feature extractor: CNN..
build CRF...


In [3]:
model.train(False)

SeqModel(
  (word_hidden): WordSequence(
    (droplstm): Dropout(p=0.4)
    (wordrep): WordRep(
      (char_feature): CharCNN(
        (char_drop): Dropout(p=0.4)
        (char_embeddings): Embedding(159, 50)
        (char_cnn): Conv1d(50, 50, kernel_size=(3,), stride=(1,), padding=(1,))
      )
      (drop): Dropout(p=0.4)
      (word_embedding): Embedding(75794, 200)
      (feature_embeddings): ModuleList()
    )
    (lstm): LSTM(250, 100, batch_first=True, bidirectional=True)
    (hidden2tag): Linear(in_features=200, out_features=24, bias=True)
  )
  (crf): CRF()
)

In [4]:
batch_size = 10
text = data.raw_Ids[:batch_size]
batch_word, batch_features, batch_wordlen, batch_wordrecover, batch_char, batch_charlen, batch_charrecover, batch_label, mask = \
batchify_with_label(text, data.HP_gpu, volatile_flag=True)
#x = torch.randn(batch_size, 1, 224, 224, requires_grad=True)
inputs = tuple((batch_word, batch_features, batch_wordlen, batch_char, batch_charlen, batch_charrecover, mask))

In [5]:
torch_out = torch.onnx._export(model,             # model being run
                               inputs,                       # model input (or a tuple for multiple inputs)
                               "super_resolution.onnx", # where to save the model (can be a file or file-like object)
                               export_params=True)  

TypeError: wrapPyFuncWithSymbolic(): incompatible function arguments. The following argument types are supported:
    1. (self: torch._C.Graph, arg0: function, arg1: List[torch::jit::Value], arg2: int, arg3: function) -> iterator

Invoked with: graph(%0 : Long(10, 42)
      %1 : Long(10)
      %2 : Long(420, 15)
      %3 : Long(420)
      %4 : Long(420)
      %5 : Byte(10, 42)
      %6 : Float(159, 50)
      %7 : Float(50, 50, 3)
      %8 : Float(50)
      %9 : Float(75794, 200)
      %10 : Float(400, 250)
      %11 : Float(400, 100)
      %12 : Float(400)
      %13 : Float(400)
      %14 : Float(400, 250)
      %15 : Float(400, 100)
      %16 : Float(400)
      %17 : Float(400)
      %18 : Float(24, 200)
      %19 : Float(24)
      %20 : Float(24, 24)) {
  %21 : Long() = aten::size[dim=0](%0), scope: SeqModel/WordSequence[word_hidden]/WordRep[wordrep]
  %22 : Long() = aten::size[dim=1](%0), scope: SeqModel/WordSequence[word_hidden]/WordRep[wordrep]
  %23 : Float(10, 42, 200) = aten::e